In [1]:
import findspark
findspark.init('/home/sdmohant/spark-3.0.0-bin-hadoop2.7')

In [2]:
import pyspark
from pyspark import SparkContext,SparkConf

In [3]:
from pyspark.sql import SparkSession

In [4]:
#spark = SparkSession.builder.appName("MyName").getOrCreate()
conf = SparkConf().set("spark.cores.max", "16") \
    .set("spark.driver.memory", "16g") \
    .set("spark.executor.memory", "16g") \
    .set("spark.executor.memory_overhead", "16g") \
    .set("spark.driver.maxResultsSize", "0") \
    .set("spark.sql.shuffle.partitions","300")

sc = SparkContext(appName="newApp", conf=conf)
spark = SparkSession(sc)

/home/sdmohant/spark-3.0.0-bin-hadoop2.7/python/pyspark/context.py:220: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)


In [5]:
from pyspark.sql.functions import isnan, count, col

In [6]:
#Read PaperAuthorAffiliations Data
AuthAndPapAuthAffMergedDF = spark.read.options(header = True,delimiter=',',inferSchema=True ).csv("AuthAndPapAuthAffMerged.csv")

In [7]:
AuthAndPapAuthAffMergedDF.show(10)

+--------+----------+-------------+----------------+----------+-------------+--------------------+
|AuthorId|AuthorRank|AutPaperCount|AutCitationCount|   PaperID|AffiliationId|AuthorSequenceNumber|
+--------+----------+-------------+----------------+----------+-------------+--------------------+
|   15437|     19253|           11|              18|1976958153|     32389192|                   1|
|   15437|     19253|           11|              18|2765447610|     32389192|                   2|
|   15437|     19253|           11|              18|1994839481|     32389192|                   5|
|   15437|     19253|           11|              18|2191263223|     32389192|                   1|
|   15437|     19253|           11|              18|2000297367|     32389192|                   5|
|   15437|     19253|           11|              18|2175552737|     32389192|                   2|
|   15437|     19253|           11|              18|2129465055|     32389192|                   5|
|   15437|

In [8]:
AuthAndPapAuthAffMergedDF.count()

223953283

In [9]:
AuthAndPapAuthAffMergedDF.select('PaperID').distinct().count()

71480858

In [10]:
AuthAndPapAuthAffMergedDF.select('AuthorRank').distinct().count()

7748

In [11]:
AuthAndPapAuthAffMergedDF.select('AuthorId').distinct().count()

33348681

In [12]:
#AuthAndPapAuthAffMergedDF = AuthAndPapAuthAffMergedDF.select('*').orderBy('PaperID').collect()

In [19]:
from pyspark.sql import Window
import pyspark.sql.functions as f
w = Window.partitionBy('PaperID')
temp_DF = AuthAndPapAuthAffMergedDF.select('PaperID', 'AuthorRank', f.min('AuthorRank').over(w).alias('min_AuthorRank')).sort('PaperID', 'AuthorRank')

In [20]:
temp_DF.show(5)

+-------+----------+--------------+
|PaperID|AuthorRank|min_AuthorRank|
+-------+----------+--------------+
|     23|     17530|         17530|
|     37|     19569|         19569|
|     37|     19697|         19569|
|    108|     19728|         19728|
|    125|     15665|         15665|
+-------+----------+--------------+
only showing top 5 rows



In [21]:
w = Window.partitionBy('PaperID')
temp_DF1 = AuthAndPapAuthAffMergedDF.select('PaperID', 'AuthorRank', f.max('AuthorRank').over(w).alias('max_AuthorRank')).sort('PaperID', 'AuthorRank')

In [22]:
temp_DF1.show(5)

+-------+----------+--------------+
|PaperID|AuthorRank|max_AuthorRank|
+-------+----------+--------------+
|     23|     17530|         17530|
|     37|     19569|         19697|
|     37|     19697|         19697|
|    108|     19728|         19728|
|    125|     15665|         19554|
+-------+----------+--------------+
only showing top 5 rows



In [23]:
w = Window.partitionBy('PaperID')
temp_DF2 = AuthAndPapAuthAffMergedDF.select('PaperID', 'AuthorRank', 'AutPaperCount', 'AutCitationCount', f.max('AuthorRank').over(w).alias('max_AuthorRank')).sort('PaperID', 'AuthorRank')

In [24]:
temp_DF2.show(2)

+-------+----------+-------------+----------------+--------------+
|PaperID|AuthorRank|AutPaperCount|AutCitationCount|max_AuthorRank|
+-------+----------+-------------+----------------+--------------+
|     23|     17530|          117|             459|         17530|
|     37|     19569|            4|               8|         19697|
+-------+----------+-------------+----------------+--------------+
only showing top 2 rows



In [25]:
w = Window.partitionBy('PaperID')
temp_DF2 = AuthAndPapAuthAffMergedDF.select('PaperID', 'AuthorRank', 'AutPaperCount', 'AutCitationCount', f.max('AuthorRank').over(w).alias('min_AuthorRank')).sort('PaperID', 'AuthorRank')

In [26]:
temp_DF2.show(2)

+-------+----------+-------------+----------------+--------------+
|PaperID|AuthorRank|AutPaperCount|AutCitationCount|min_AuthorRank|
+-------+----------+-------------+----------------+--------------+
|     23|     17530|          117|             459|         17530|
|     37|     19569|            4|               8|         19697|
+-------+----------+-------------+----------------+--------------+
only showing top 2 rows



In [28]:
w = Window.partitionBy('PaperID')
temp_DF2 = temp_DF2.select('PaperID', 'AuthorRank', 'AutPaperCount', 'AutCitationCount', 'min_AuthorRank', f.max('AuthorRank').over(w).alias('max_AuthorRank')).sort('PaperID', 'AuthorRank')

In [29]:
temp_DF2.show(2)

+-------+----------+-------------+----------------+--------------+--------------+
|PaperID|AuthorRank|AutPaperCount|AutCitationCount|min_AuthorRank|max_AuthorRank|
+-------+----------+-------------+----------------+--------------+--------------+
|     23|     17530|          117|             459|         17530|         17530|
|     37|     19569|            4|               8|         19697|         19697|
+-------+----------+-------------+----------------+--------------+--------------+
only showing top 2 rows



In [32]:
temp_DF2 = temp_DF2.select('PaperID', 'AuthorRank', 'AutPaperCount', 'AutCitationCount', 'min_AuthorRank','max_AuthorRank', f.min('AutPaperCount').over(w).alias('min_AutPaperCount')).sort('PaperID', 'AuthorRank')

In [33]:
temp_DF2 = temp_DF2.select('PaperID', 'AuthorRank', 'AutPaperCount', 'AutCitationCount', 'min_AuthorRank','max_AuthorRank','min_AutPaperCount', f.max('AutPaperCount').over(w).alias('max_AutPaperCount')).sort('PaperID', 'AuthorRank')

In [34]:
temp_DF2 = temp_DF2.select('PaperID', 'AuthorRank', 'AutPaperCount', 'AutCitationCount', 'min_AuthorRank','max_AuthorRank','min_AutPaperCount','max_AutPaperCount', f.min('AutCitationCount').over(w).alias('min_AutCitationCount')).sort('PaperID', 'AuthorRank')

In [35]:
temp_DF2 = temp_DF2.select('PaperID', 'AuthorRank', 'AutPaperCount', 'AutCitationCount', 'min_AuthorRank','max_AuthorRank','min_AutPaperCount','max_AutPaperCount', 'min_AutCitationCount', f.max('AutCitationCount').over(w).alias('max_AutCitationCount')).sort('PaperID', 'AuthorRank')

In [36]:
temp_DF2.show(10)

+-------+----------+-------------+----------------+--------------+--------------+-----------------+-----------------+--------------------+--------------------+
|PaperID|AuthorRank|AutPaperCount|AutCitationCount|min_AuthorRank|max_AuthorRank|min_AutPaperCount|max_AutPaperCount|min_AutCitationCount|max_AutCitationCount|
+-------+----------+-------------+----------------+--------------+--------------+-----------------+-----------------+--------------------+--------------------+
|     23|     17530|          117|             459|         17530|         17530|              117|              117|                 459|                 459|
|     37|     19569|            4|               8|         19697|         19697|                2|                4|                   3|                   8|
|     37|     19697|            2|               3|         19697|         19697|                2|                4|                   3|                   8|
|    108|     19728|            1|      

In [38]:
temp_DF2 = temp_DF2.select('PaperID', 'AuthorRank', 'AutPaperCount', 'AutCitationCount', 'max_AuthorRank','min_AutPaperCount','max_AutPaperCount', 'min_AutCitationCount','max_AutCitationCount', f.min('AuthorRank').over(w).alias('min_AuthorRank')).sort('PaperID', 'AuthorRank')

In [40]:
temp_DF2.show(10)

+-------+----------+-------------+----------------+--------------+-----------------+-----------------+--------------------+--------------------+--------------+
|PaperID|AuthorRank|AutPaperCount|AutCitationCount|max_AuthorRank|min_AutPaperCount|max_AutPaperCount|min_AutCitationCount|max_AutCitationCount|min_AuthorRank|
+-------+----------+-------------+----------------+--------------+-----------------+-----------------+--------------------+--------------------+--------------+
|     23|     17530|          117|             459|         17530|              117|              117|                 459|                 459|         17530|
|     37|     19569|            4|               8|         19697|                2|                4|                   3|                   8|         19569|
|     37|     19697|            2|               3|         19697|                2|                4|                   3|                   8|         19569|
|    108|     19728|            1|      

In [46]:
drop_cols = ['AuthorRank','AutPaperCount','AutCitationCount']
temp_DF2 = temp_DF2.drop(*drop_cols)

In [47]:
temp_DF2.show(2)

+-------+--------------+-----------------+-----------------+--------------------+--------------------+--------------+
|PaperID|max_AuthorRank|min_AutPaperCount|max_AutPaperCount|min_AutCitationCount|max_AutCitationCount|min_AuthorRank|
+-------+--------------+-----------------+-----------------+--------------------+--------------------+--------------+
|     23|         17530|              117|              117|                 459|                 459|         17530|
|     37|         19697|                2|                4|                   3|                   8|         19569|
+-------+--------------+-----------------+-----------------+--------------------+--------------------+--------------+
only showing top 2 rows



In [48]:
PapJouCSCI = spark.read.options(header = True, inferSchema = True,delimiter=',').csv("PapJournalCSCI.csv")

In [49]:
PapJouCSCI.show(2)

+--------------+------------+---------+----------+----------+------------+-------------+--------------+------------+---------------+---------------+------+------------+----------+------+------------+------------------+------------+
|ConfInstanceID|ConfSeriesID|JournalID|   PaperID|Paper_Rank|Paper_RefCnt|CitationCount|EstimatedCount|Journal_Rank|Journal_Pap_Cnt|Journal_Cit_Cnt|CSRank|CSPaperCount|CSCitation|CIRank|CIPaperCount|CIPaperCitationCnt|DocTypeIndex|
+--------------+------------+---------+----------+----------+------------+-------------+--------------+------------+---------------+---------------+------+------------+----------+------+------------+------------------+------------+
|    2735348883|  2734834919| 23074404|1819143627|     24199|           0|            0|             0|        9728|           9284|          64351| 11187|        4309|      6256| 25000|         405|               759|         0.0|
|     180417304|  1141689323|137519996|1963578506|     22929|          1

In [50]:
drop_cols = ['ConfInstanceID','ConfSeriesID','JournalID']
PapJouCSCI = PapJouCSCI.drop(*drop_cols)

In [51]:
PapJouCSCI.count()

90728

In [54]:
temp_DF2.selectdistinct().count()

71480858

In [60]:
temp_DF2.select("PaperID").distinct().count()

71480858

In [61]:
final_df = PapJouCSCI.join(temp_DF2, on=['PaperID'], how='inner')

In [62]:
final_df.count()

266425

In [66]:
final_df1 =  final_df.dropDuplicates()

In [67]:
final_df1.count()

82711

In [68]:
final_df1.coalesce(1).write.format('com.databricks.spark.csv').mode("overwrite").save('AutPapJouCSCI.csv',header = 'true')